In [22]:
import psycopg2

# Connect to the database
conn = psycopg2.connect("dbname=hn user=julien")

# Open a cursor to perform database operations
cur = conn.cursor()

# Query the database and obtain data as Python objects
cur.execute("SELECT max(id) FROM hn_post LIMIT 1")
maxDB = cur.fetchone()[0]



In [23]:
# Max HN ID post
import aiohttp

requests = aiohttp.ClientSession()

async def get_max_id():
    async with requests.get("https://hacker-news.firebaseio.com/v0/maxitem.json") as response:
        return await response.json()
    
max_id = await get_max_id()

Unclosed client session
client_session: <aiohttp.client.ClientSession object at 0x10a3b0700>
Unclosed connector
connections: ['[(<aiohttp.client_proto.ResponseHandler object at 0x10db1d660>, 453374.922926458), (<aiohttp.client_proto.ResponseHandler object at 0x10db1e260>, 453374.949180833), (<aiohttp.client_proto.ResponseHandler object at 0x10db1cb80>, 453374.959075416), (<aiohttp.client_proto.ResponseHandler object at 0x10db1c9a0>, 453374.970496625), (<aiohttp.client_proto.ResponseHandler object at 0x10ab8f2e0>, 453374.979416416), (<aiohttp.client_proto.ResponseHandler object at 0x10db1f880>, 453374.980096291), (<aiohttp.client_proto.ResponseHandler object at 0x105845420>, 453374.985647083), (<aiohttp.client_proto.ResponseHandler object at 0x10db62140>, 453374.989837333), (<aiohttp.client_proto.ResponseHandler object at 0x10abe3a60>, 453374.998967041), (<aiohttp.client_proto.ResponseHandler object at 0x10ae09c60>, 453375.007842416), (<aiohttp.client_proto.ResponseHandler object at 0x1

In [24]:
""" 
-- Table Definition
CREATE TABLE "public"."hn_post" (
    "id" int4 NOT NULL,
    "type" text NOT NULL,
    "by" text,
    "time" int8,
    "text" text,
    "parent" int4,
    "kids" _text,
    "url" text,
    "descendants" int4,
    "score" int4,
    "title" text,
    "deleted" bool,
    "dead" bool,
    PRIMARY KEY ("id")
);
"""

async def insert_hn_post(id):
    try:
        async with requests.get(f"https://hacker-news.firebaseio.com/v0/item/{id}.json") as response:
            post = await response.json()
            if post and post.get("type") == "story":
                if post.get("title") == "" or post.get("title") is None:
                    return
                
                try:
                    cur.execute(
                        """INSERT INTO hn_post (id, type, by, time, text, parent, kids, url, descendants,
                        score, title, deleted, dead) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)""",
                        (post.get("id"), post.get("type"), post.get("by"), post.get("time"), post.get("text"),
                            post.get("parent"), post.get("kids"), post.get("url"), post.get("descendants"),
                            post.get("score"), post.get("title"), post.get("deleted"), post.get("dead"))
                    )
                    conn.commit()
                    print("Done inserting {}: {}".format(id, post.get("title")))
                except Exception as e:
                    conn.rollback()
                    print(e)
    except Exception as e:
        print(e)



In [25]:
import aiometer
ids= [i for i in range(maxDB, max_id)]
print(len(ids))

await aiometer.run_on_each(insert_hn_post, ids, max_per_second=100)

14
duplicate key value violates unique constraint "hn_post_pkey"
DETAIL:  Key (id)=(38981765) already exists.

Done inserting 38981777: Lawsuit: Hudson Co Jail Recorded Calls with Attorneys, Gave Info to Prosecutors
